In [5]:
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch

# Load the dataset software requirements label:text
data = []
with open('C:/Users/dzyyu/Arbeit/MasterArbeit/MASTERARBEIT/Datasets/classification/Software/software_requirements.txt', 'r') as file:
    for line in file:
        if ':' in line:
            label, text = line.strip().split(':', 1)
            data.append({'label': label, 'text': text})

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'][:,0,:].numpy()

# Apply BERT encoding to the text
df['embedding'] = df['text'].apply(get_bert_embedding)

# Flatten the embeddings
X = np.vstack(df['embedding'].to_numpy())

# Convert labels into numeric format
le = LabelEncoder()
y = le.fit_transform(df['label'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TPOT classifier without custom configuration
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, max_time_mins=15)

# Fit the classifier on the training data
tpot.fit(X_train, y_train)

# Evaluate the model
print(tpot.score(X_test, y_test))

# Export the best pipeline
tpot.export('tpot_2008_pipeline.py')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Version 0.12.0 of tpot is outdated. Version 0.12.1 was released Tuesday August 15, 2023.


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7583333333333334

Generation 2 - Current best internal CV score: 0.7583333333333334

Generation 3 - Current best internal CV score: 0.775

Generation 4 - Current best internal CV score: 0.775

Generation 5 - Current best internal CV score: 0.775

Best pipeline: MLPClassifier(input_matrix, alpha=0.0001, learning_rate_init=0.01)
0.5333333333333333


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score

#prediction with TPOT. Bert encoding is done manually. For the prediction used, the algorithm chosen by TPOT

#load the data
data = []
with open('C:/Users/dzyyu/Arbeit/MasterArbeit/MASTERARBEIT/Datasets/classification/Software/software_requirements.txt', 'r') as file:
    for line in file:
        if ':' in line:
            label, text = line.strip().split(':', 1)
            data.append({'label': label, 'text': text})

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

#Bert encoding
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'][:,0,:].numpy()

# Apply BERT encoding to the text
df['embedding'] = df['text'].apply(get_bert_embedding)

# Flatten the embeddings
X = np.vstack(df['embedding'].to_numpy())

# Convert labels into numeric format
le = LabelEncoder()
y = le.fit_transform(df['label'])

tpot_data=df
# NOTE: Make sure that the outcome column is labeled 'target' in the data file
#tpot_data = pd.read_csv('C:/Users/dzyyu/Arbeit/MasterArbeit/MASTERARBEIT/Datasets/classification/Software/test.txt', sep=':', dtype=np.float64)
#features = tpot_data.drop('label', axis=1)

training_features, testing_features, training_target, testing_target = \
            train_test_split(X, y, random_state=None)
          
exported_pipeline = MLPClassifier(alpha=0.0001, learning_rate_init=0.01)

#training the model chosen by TPOT
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
print(results)

#decode results
decoded_results = le.inverse_transform(results)
print(decoded_results)
accuracy = accuracy_score(testing_target, results)
print(f"Accuracy Score: {accuracy * 100:.2f}%")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[ 1  1 11  1  7 11  1 10  4  1 11 11  1  1 11  1  1  0  1  6  1  9  1 11
  1 11  1  1  1  1 11  7  6  6  6  1  1  1]
['F' 'F' 'US' 'F' 'PE' 'US' 'F' 'SE' 'LF' 'F' 'US' 'US' 'F' 'F' 'US' 'F'
 'F' 'A' 'F' 'O' 'F' 'SC' 'F' 'US' 'F' 'US' 'F' 'F' 'F' 'F' 'US' 'PE' 'O'
 'O' 'O' 'F' 'F' 'F']
Accuracy Score: 84.21%
